In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import numpy as np
import os
import re
import scipy as sci
import time

In [2]:
# global vars
samp_rt = 5120 #sampling rate Hz
time_import = 7200 # time to analyse, seconds
data_import = samp_rt * time_import # lines of data to import

In [3]:
def list_o_list(big_list, lil_size):
    l_o_l = []
    for i in range(0, len(big_list), lil_size):
        lil = big_list[i : min(i + lil_size, len(big_list))]
        l_o_l.append(lil)
    return(l_o_l)

In [4]:
def freq_analysis(sample , ttime, responsefrq =1, sampfrq = 5120):
#     mmtic = time.perf_counter()
    frqs = ""
    amps = []
    samp_int = 1/sampfrq
    ft = np.fft.fft(sample)/len(sample)
#     mmhic = time.perf_counter()
    ft = ft[range(int(len(sample)/2))]
    tpCount = len(sample)
    values = np.arange(int(tpCount/2))
    timePeriod = tpCount/sampfrq
    frequencies = values/timePeriod
    ft = abs(ft)
#     mmcup = time.perf_counter()
    #print(len(ft))
    #print(len(frequencies))
    feq = pd.DataFrame(data = {"freq":frequencies})
#     l1 = time.perf_counter()
    feq["amp"] = ft
#     l2 = time.perf_counter()
    frqs = np.arange(0, 1 + int(feq.freq.max()), responsefrq)
#     l3 = time.perf_counter()
    #print(len(frqs))
#     mmbless = time.perf_counter()
    wow = feq.amp.rolling(responsefrq).mean()
    yep = np.arange(responsefrq - 1, len(wow) + responsefrq  -1, responsefrq)
    amps = wow[yep]
#     l4 = time.perf_counter()
#     mmtoc = time.perf_counter()
#     print("init:", mmhic-mmtic)
#     print("fft:", mmcup - mmhic, "total:", mmcup - mmtic)
#     print("df1:", mmbless - mmcup, "total:", mmbless - mmtic)
#     print("done:", mmtoc - mmbless, "total:", mmtoc - mmtic)
#     print("l1, l2, l3, l4:", l1-mmcup, l2-mmcup, l3-mmcup, l4-mmcup)
#     print("for:", l4 - l3)
    return(frqs, amps)

In [5]:
16 // 5



3

In [6]:
def process(file):
    df = pd.read_csv(file.path, header=15, names=["time", "acceleration"])
    readings = len(df)
    transformable_readings  = readings // samp_rt
    max_lines = (transformable_readings * samp_rt)
    df2 = df.iloc[:(max_lines -1),:]
#     df2 = df.iloc[:data_import,:]
    l1 = list(df2.acceleration)
    
    print("l1", len(l1))
    l2 = list_o_list(l1, 5120)
    del l1
    print("l2", len(l2))
    bin_size = 10
    tot_frqs = int((samp_rt / 2) / bin_size)
    tic_a = time.perf_counter()
    ft_l1 = np.empty((transformable_readings, tot_frqs))
#     ft_l1 = np.empty((time_import, tot_frqs))
    frq_l1 = np.empty(tot_frqs, np.int8)
    try:
        for c,v in enumerate(l2):
            frq_l1, amp = freq_analysis(v, 1, bin_size, samp_rt) # 1 is the total time of the sample (1 second)
            fail_c = c
            ft_l1[c] = amp
    except:
        print(fail_c, fail_c/len(l2))
    toc_a = time.perf_counter()
    print("time diff:", toc_a - tic_a)
    direct = pd.DataFrame(data = ft_l1, columns = frq_l1)
    l2_df = pd.DataFrame(data =l2)
    del l2
    direct["mean_acc"] = l2_df.abs().mean(axis=1)
    direct["median"] = l2_df.abs().median(axis=1)
    direct["95_acc"] = l2_df.T.abs().quantile(.95)
    direct["max_acc"] = l2_df.abs().max(axis=1)
    del l2_df
    file_name = file.name[:-3] + "big"
    direct.to_csv(file_name)
    toe = time.perf_counter()
    print("Finished in:", toe - tic_a, "seconds")
    return()
    
    
    
    
    
    

In [7]:
os.listdir()
home = os.getcwd()
dirs = []
dirs.append(home)
with os.scandir(home) as contents:
    for i in contents:
        if i.is_dir():
            dirs.append(i.path)
print(dirs)
                 


['C:\\Users\\work_cbdvl\\Documents\\GitHub\\sturm', 'C:\\Users\\work_cbdvl\\Documents\\GitHub\\sturm\\.git', 'C:\\Users\\work_cbdvl\\Documents\\GitHub\\sturm\\.ipynb_checkpoints', 'C:\\Users\\work_cbdvl\\Documents\\GitHub\\sturm\\01-08-19-V118', 'C:\\Users\\work_cbdvl\\Documents\\GitHub\\sturm\\03-05-19-V118', 'C:\\Users\\work_cbdvl\\Documents\\GitHub\\sturm\\03-26-19-V118', 'C:\\Users\\work_cbdvl\\Documents\\GitHub\\sturm\\08-14-18-V118', 'C:\\Users\\work_cbdvl\\Documents\\GitHub\\sturm\\gps_files', 'C:\\Users\\work_cbdvl\\Documents\\GitHub\\sturm\\Rec1', 'C:\\Users\\work_cbdvl\\Documents\\GitHub\\sturm\\Rec2', 'C:\\Users\\work_cbdvl\\Documents\\GitHub\\sturm\\Rec3', 'C:\\Users\\work_cbdvl\\Documents\\GitHub\\sturm\\Rec4', 'C:\\Users\\work_cbdvl\\Documents\\GitHub\\sturm\\Rec5', 'C:\\Users\\work_cbdvl\\Documents\\GitHub\\sturm\\Rec6', 'C:\\Users\\work_cbdvl\\Documents\\GitHub\\sturm\\Test Train Sample Noise Data']


In [8]:
for i in dirs:
    
    with os.scandir(i) as stuff:
        for thing in stuff:
            if re.search(".csv", thing.name) and re.search("_ch\d", thing.name):
                print(thing.name)
                try:
                    process(thing)
                except:
                    print(thing.name, "- FAILED!")
    


01-08-19-V118_ch1.csv
l1 51353599
l2 10030
10028 0.9998005982053838
time diff: 35.13362219999999
Finished in: 68.35619870000005 seconds
01-08-19-V118_ch2.csv
l1 51353599
l2 10030
10028 0.9998005982053838
time diff: 34.57260689999998
Finished in: 68.04574830000001 seconds
03-05-19_ch1.csv
l1 59740159
l2 11668
11666 0.9998285910181693
time diff: 42.038704099999904
Finished in: 81.15499640000007 seconds
03-05-19_ch2.csv
l1 59740159
l2 11668
11666 0.9998285910181693
time diff: 41.62022550000006
Finished in: 81.33561039999995 seconds
03-26-19_ch1.csv
l1 51788799
l2 10115
10113 0.9998022738507167
time diff: 44.74612819999993
Finished in: 79.12638109999989 seconds
03-26-19_ch2.csv
l1 51788799
l2 10115
10113 0.9998022738507167
time diff: 36.03200059999995
Finished in: 72.36068739999996 seconds
08-14-18-V118_ch1.csv
l1 31728639
l2 6197
6195 0.999677263191867
time diff: 22.53988709999976
Finished in: 42.33335309999984 seconds
08-14-18-V118_ch2.csv
l1 31728639
l2 6197
6195 0.999677263191867
time 